In [2]:
import matplotlib
matplotlib.use("Agg")
# import the necessary packages

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

In [15]:
print("[INFO] loading images...")
data = []
labels = []

# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images("images")))
random.seed(42)
random.shuffle(imagePaths)

[INFO] loading images...


In [17]:

# loop over the input images
for imagePath in imagePaths:
    # load the image, resize the image to be 32x32 pixels (ignoring
    # aspect ratio), flatten the image into 32x32x3=3072 pixel image
    # into a list, and store the image in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (32, 32)).flatten()
    data.append(image)
    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

In [18]:
data

[array([169, 156, 134, ..., 158, 190, 203], dtype=uint8),
 array([181, 198, 211, ..., 135, 158, 170], dtype=uint8),
 array([ 56, 147,  57, ...,  80,  60,  55], dtype=uint8),
 array([169, 156, 134, ..., 158, 190, 203], dtype=uint8),
 array([181, 198, 211, ..., 135, 158, 170], dtype=uint8),
 array([ 56, 147,  57, ...,  80,  60,  55], dtype=uint8)]

In [19]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [20]:
data.shape

(6, 3072)

In [23]:
labels

array(['images', 'images', 'images', 'images', 'images', 'images'],
      dtype='<U6')

In [24]:
(trainX, testX, trainY, testY) = train_test_split(data,
labels, test_size=0.25, random_state=42)

In [25]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)


In [29]:
# define the 3072-1024-512-3 architecture using Keras
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))


# initialize our initial learning rate and # of epochs to train for
INIT_LR = 0.01
EPOCHS = 80

In [34]:
print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
metrics=["accuracy"])



# train the neural network
H = model.fit(x=trainX, y=trainY, validation_data=(testX, testY),
epochs=EPOCHS, batch_size=32)

[INFO] training network...
Train on 4 samples, validate on 2 samples
Epoch 1/80
4/4 [==============================] - 1s 177ms/sample - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/80
4/4 [==============================] - 0s 10ms/sample - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/80
4/4 [==============================] - 0s 10ms/sample - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/80
4/4 [==============================] - 0s 10ms/sample - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/80
4/4 [==============================] - 0s 10ms/sample - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/80
4/4 [==============================] - 0s 10ms/sample - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy:

In [36]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
predictions.argmax(axis=1), target_names=lb.classes_))
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()

[INFO] evaluating network...
              precision    recall  f1-score   support

      images       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



In [37]:
# save the model and label binarizer to disk
print("[INFO] serializing network and label binarizer...")
model.save("anaconda.h5")
f = open(args["label_bin"], "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] serializing network and label binarizer...


NameError: name 'args' is not defined